In [ ]:
# default_exp data_processing
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import FRED
if torch.__version__[:4] == '1.13': # If using pytorch with MPS, use Apple silicon GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.has_mps else "cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
%load_ext autoreload
%autoreload 2

Using device mps


# 02d Flow Datasets
> Here be the tailored datasets for FRED! In addition to storing points, we must store a graph, and intelligently sample from the graph so that batches preserve 'flow neighborhoods'

As previously listed, this is our preprocessing pipeline: 
0-3 are handled by separate functions. This notebook takes care of 4 and 5

0. Turn data to torch tensors
1. Build graph from data, using flashlight kernel
	1. Estimate best parameters for kernel selection
2. Take diffusion map on data (or any ‘best guess embedding’) we want to penalize against. (Can be done on CPU or GPU). Would likely use established methods, like PHATE, tSNE, node2vec.
	1. Visualize embedding
3. Find flow neighborhoods of data.
4. **Break data into batches (both affinity matrix of batches and points in batches).**
5. **Return `Dataloader` with minibatch `X` and subgraph `A` of each batch.**